In [1]:
import oandapyV20
# from oandapyV20 import API
# import oandapyV20.endpoints.trades as trades
import oandapyV20.endpoints.accounts as accounts
# import oandapyV20.endpoints.instruments as instruments
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
import configparser

import cred
import pandas as pd
import json

import datetime
# api = API(access_token="...")

### Getting all the instruments to load into the Symbol Table

In [2]:
accountID=cred.acc_id_prac
access_token=cred.token_prac
api = oandapyV20.API(access_token=access_token)
client = oandapyV20.API(access_token=access_token)

r = accounts.AccountInstruments(accountID=accountID)
rv = client.request(r)
print(json.dumps(rv, indent=2))


{
  "instruments": [
    {
      "name": "USD_ZAR",
      "type": "CURRENCY",
      "displayName": "USD/ZAR",
      "pipLocation": -4,
      "displayPrecision": 5,
      "tradeUnitsPrecision": 0,
      "minimumTradeSize": "1",
      "maximumTrailingStopDistance": "1.00000",
      "minimumTrailingStopDistance": "0.00050",
      "maximumPositionSize": "0",
      "maximumOrderUnits": "100000000",
      "marginRate": "0.07",
      "tags": [
        {
          "type": "ASSET_CLASS",
          "name": "CURRENCY"
        }
      ]
    },
    {
      "name": "GBP_HKD",
      "type": "CURRENCY",
      "displayName": "GBP/HKD",
      "pipLocation": -4,
      "displayPrecision": 5,
      "tradeUnitsPrecision": 0,
      "minimumTradeSize": "1",
      "maximumTrailingStopDistance": "1.00000",
      "minimumTrailingStopDistance": "0.00050",
      "maximumPositionSize": "0",
      "maximumOrderUnits": "100000000",
      "marginRate": "0.05",
      "tags": [
        {
          "type": "ASSET_CLASS",

In [8]:
df=pd.read_json(json.dumps(rv, indent=2))
df=pd.io.json.json_normalize(data=df['instruments'])

In [9]:
df.head()

,displayName,displayPrecision,marginRate,maximumOrderUnits,maximumPositionSize,maximumTrailingStopDistance,minimumTradeSize,minimumTrailingStopDistance,name,pipLocation,tags,tradeUnitsPrecision,type
0,USD/ZAR,5,0.07,100000000,0,1.00000,1,0.00050,USD_ZAR,-4,"[{'type': 'ASSET_CLASS', 'name': 'CURRENCY'}]",0,CURRENCY
1,GBP/HKD,5,0.05,100000000,0,1.00000,1,0.00050,GBP_HKD,-4,"[{'type': 'ASSET_CLASS', 'name': 'CURRENCY'}]",0,CURRENCY
2,EUR/GBP,5,0.05,100000000,0,1.00000,1,0.00050,EUR_GBP,-4,"[{'type': 'ASSET_CLASS', 'name': 'CURRENCY'}]",0,CURRENCY
3,GBP/SGD,5,0.05,100000000,0,1.00000,1,0.00050,GBP_SGD,-4,"[{'type': 'ASSET_CLASS', 'name': 'CURRENCY'}]",0,CURRENCY
4,USD/SEK,5,0.03,100000000,0,1.00000,1,0.00050,USD_SEK,-4,"[{'type': 'ASSET_CLASS', 'name': 'CURRENCY'}]",0,CURRENCY


In [10]:
symbols=[]
now = datetime.datetime.utcnow()
for i, symbol in df.iterrows():
#     print(symbol)
    symbols.append(
                    (symbol['name'],'equity',
                     symbol['displayName'],
                     'Forex', 'USD', now, now)
                  )

### Getting the historical data to load into the Daily Table

In [19]:
def oanda_historical_data(instrument,granularity='D',start_date="2019-01-01T00:00:00Z",end_date="2019-05-01T00:00:00Z"):
    params = {
    "from": start_date,
    "to": end_date,
    "granularity": granularity,
    "count": 2500,
    }

    df_full=pd.DataFrame()
    for r in InstrumentsCandlesFactory(instrument=instrument,params=params):
        client.request(r)
        dat = []
        api_data=r.response.get('candles')
        if(api_data):
            for oo in r.response.get('candles'):
                dat.append([oo['time'], oo['volume'], oo['mid']['o'], oo['mid']['h'], oo['mid']['l'], oo['mid']['c']])
                
            df = pd.DataFrame(dat)
            df.columns = ['time', 'volume', 'open', 'high', 'low', 'close']
            df = df.set_index('time')
            if df_full.empty:
                df_full=df
            else:
                df_full=df_full.append(df)
    df_full.index=pd.to_datetime(df_full.index)    
    return df_full

In [20]:
# %%time
df_market=oanda_historical_data(instrument="EUR_USD",granularity='D',start_date="2019-01-01T00:00:00Z") #start_dt.strftime("%Y-%m-%dT%H:%M:%SZ")

In [24]:
df_market.head()

,volume,open,high,low,close
time,,,,,
2019-01-01 22:00:00+00:00,35508,1.14649,1.14970,1.13250,1.13447
2019-01-02 22:00:00+00:00,54135,1.13431,1.14112,1.13091,1.13942
2019-01-03 22:00:00+00:00,41022,1.13930,1.14190,1.13458,1.13951
2019-01-06 22:00:00+00:00,25648,1.13974,1.14827,1.13972,1.14748
2019-01-07 22:00:00+00:00,25474,1.14773,1.14848,1.14219,1.14417


In [22]:
df_market.index[-1]

Timestamp('2019-04-30 21:00:00+0000', tz='UTC')

## Extra

In [ ]:
accountID=cred.acc_id_prac
access_token=cred.token_prac

api = API(access_token=access_token)

In [ ]:
client = oandapyV20.API(access_token=access_token)
r = accounts.AccountDetails(accountID)

In [ ]:
params = {
          "count": 10,
          "granularity": "M1"}

In [ ]:
r = instruments.InstrumentsCandles(instrument="EUR_USD",
                                   params=params)

In [ ]:
r.response.get('candles')

In [ ]:
type(client.request(r))

In [ ]:
dat = []
for oo in r.response['candles']:
    dat.append([oo['time'], oo['volume'], oo['mid']['o'], oo['mid']['h'], oo['mid']['l'], oo['mid']['c']])

In [ ]:
df = pd.DataFrame(dat)
df.columns = ['Time', 'Volume', 'Open', 'High', 'Low', 'Close']
df = df.set_index('Time')
df.head()

In [ ]:
r.response['candles']

In [ ]:
import json
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory

instrument, granularity = "EUR_USD", "M1"
_from = "2019-01-01T00:00:00Z"
params = {
"from": _from,
"granularity": granularity,
"count": 2500,
}

df_full=pd.DataFrame()
for r in InstrumentsCandlesFactory(instrument=instrument,params=params):
    client.request(r)
    dat = []
    for oo in r.response.get('candles'):
        dat.append([oo['time'], oo['volume'], oo['mid']['o'], oo['mid']['h'], oo['mid']['l'], oo['mid']['c']])
    df = pd.DataFrame(dat)
    df.columns = ['Time', 'Volume', 'Open', 'High', 'Low', 'Close']
    df = df.set_index('Time')
    if df_full.empty:
        df_full=df
    else:
        df_full=df_full.append(df)

In [41]:
df_market.head()

,volume,open,high,low,close
time,,,,,
2019-01-01 22:00:00+00:00,35508,1.14649,1.14970,1.13250,1.13447
2019-01-02 22:00:00+00:00,54135,1.13431,1.14112,1.13091,1.13942
2019-01-03 22:00:00+00:00,41022,1.13930,1.14190,1.13458,1.13951
2019-01-06 22:00:00+00:00,25648,1.13974,1.14827,1.13972,1.14748
2019-01-07 22:00:00+00:00,25474,1.14773,1.14848,1.14219,1.14417


In [60]:
(df_market.reset_index()['time'][1:].reset_index()['time']-df_market.reset_index()['time']).max()

Timedelta('3 days 03:01:00')

In [55]:
df_market.reset_index()['time'][1:].reset_index()['time'].head()

0   2019-01-01 22:01:00+00:00
1   2019-01-01 22:02:00+00:00
2   2019-01-01 22:03:00+00:00
3   2019-01-01 22:05:00+00:00
4   2019-01-01 22:10:00+00:00
Name: time, dtype: datetime64[ns, UTC]

In [49]:
df_market.reset_index()['time'][1:]

1       2019-01-01 22:01:00+00:00
2       2019-01-01 22:02:00+00:00
3       2019-01-01 22:03:00+00:00
4       2019-01-01 22:05:00+00:00
5       2019-01-01 22:10:00+00:00
6       2019-01-01 22:11:00+00:00
7       2019-01-01 22:15:00+00:00
8       2019-01-01 22:24:00+00:00
9       2019-01-01 22:26:00+00:00
10      2019-01-01 22:27:00+00:00
11      2019-01-01 22:28:00+00:00
12      2019-01-01 22:29:00+00:00
13      2019-01-01 22:30:00+00:00
14      2019-01-01 22:31:00+00:00
15      2019-01-01 22:32:00+00:00
16      2019-01-01 22:33:00+00:00
17      2019-01-01 22:34:00+00:00
18      2019-01-01 22:35:00+00:00
19      2019-01-01 22:36:00+00:00
20      2019-01-01 22:37:00+00:00
21      2019-01-01 22:38:00+00:00
22      2019-01-01 22:39:00+00:00
23      2019-01-01 22:40:00+00:00
24      2019-01-01 22:41:00+00:00
25      2019-01-01 22:43:00+00:00
26      2019-01-01 22:44:00+00:00
27      2019-01-01 22:45:00+00:00
28      2019-01-01 22:46:00+00:00
29      2019-01-01 22:47:00+00:00
30      2019-0

In [27]:
df_market['time_']=df_market.reset_index()['time']
df_market['time_1D_fwd']=df_market.shift().reset_index()['time']

In [28]:
df_market.head()

,volume,open,high,low,close,time_,time_1D_fwd
time,,,,,,,
2019-01-01 22:00:00+00:00,1,1.14649,1.14649,1.14649,1.14649,NaT,NaT
2019-01-01 22:01:00+00:00,1,1.14634,1.14634,1.14634,1.14634,NaT,NaT
2019-01-01 22:02:00+00:00,2,1.14634,1.14634,1.14634,1.14634,NaT,NaT
2019-01-01 22:03:00+00:00,1,1.14658,1.14658,1.14658,1.14658,NaT,NaT
2019-01-01 22:05:00+00:00,2,1.14658,1.14658,1.14635,1.14635,NaT,NaT


In [ ]:
type(df_full['Close'])

In [ ]:
df_full.index

In [ ]:
pd.to_datetime(df_full.index)

In [ ]:
df_full=df_full.astype(float)
df_full['Open'].plot()

In [ ]:
import oandapyV20.endpoints.accounts as accounts
r = accounts.AccountInstruments(accountID=accountID)
client.request(r)
pd.DataFrame(test['instruments'])['displayName']